In [ ]:
from google.colab import files
files.upload()  # Pilih file `kaggle.json` dari komputer Anda

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

# Buat folder .kaggle jika belum ada
os.makedirs('/root/.kaggle', exist_ok=True)

# Pindahkan file ke /root/.kaggle/
!mv kaggle.json /root/.kaggle/

# Berikan izin akses yang benar
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
from kaggle.api.kaggle_api_extended import KaggleApi

api = KaggleApi()
api.authenticate()

# Unduh dataset dan simpan di direktori 'dataset/'
api.dataset_download_files('kaustubhb999/tomatoleaf', path='dataset/', unzip=True)

In [ ]:
import os
os.listdir('dataset/')

In [ ]:
!kaggle datasets list

In [ ]:
import os

# Menampilkan struktur folder di dalam /content/dataset
for root, dirs, files in os.walk("/content/dataset"):
    print("Folder:", root)
    print("Sub-folder:", dirs)
    print("File:", files)
    print("-" * 50)


In [ ]:
#Import library awal yang diperlukan
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import skimage.io
import tensorflow as tf
import cv2
import tqdm
import glob
%matplotlib inline

In [ ]:
#Mengecek apakah ada gambar yang rusak pada folder Validation
from PIL import Image
from pathlib import Path

for file_path in Path('/content/dataset/tomato/val').glob('**/*.jpg'):
    print('Check: %s' % file_path)

    im = Image.open(file_path)
    im.verify()

    print('Done.')

In [ ]:
#Menampilkan Contoh Gambar dari Dataset
plt.figure(figsize=(12,8))

a=0
temp = []
for i in os.listdir('/content/dataset/tomato/train'): #Akses subdirektori di dalam folder train
  path =  os.path.join('/content/dataset/tomato/train',i)
  for j in os.listdir(path): #Membaca file di dalam setiap subdirektori
    path2 = os.path.join(path,j)
    a+=1
    plt.subplot(2,5,a)
    plt.imshow(cv2.imread(path2))
    plt.title(i, fontsize=7)
    plt.axis('off')

    break

print(a)

In [ ]:
#Mengabaikan data yang rusak
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#Mendefenisikan fungsi fungsi pada augmentasi data, fungsi yang digunakan adalah melakukan normalisasi gambar
train_datagen = ImageDataGenerator(rescale = 1.0 / 255.0)

valid_datagen = ImageDataGenerator(rescale = 1.0 / 255.0)

In [ ]:
#Mendefenisikan data train dan kemudian melakukan augmentasi
train_dataset  = train_datagen.flow_from_directory(directory = '/content/dataset/tomato/train',#Direktori data train
                                                   target_size = (224,224),#Dimensi gambar (Panjang x Lebar)
                                                   class_mode = 'categorical',#Mendefinisikan mode kelas (Categorical berarti kelas akan dibentuk kedalam one-hot-encoding)
                                                   batch_size = 32)#Mendefinisikan batch size

In [ ]:
#Mendefenisikan data validation dan kemudian melakukan augmentasi
val_dataset  = valid_datagen.flow_from_directory(directory = '/content/dataset/tomato/val',#Direktori data validation
                                                   target_size = (224,224),
                                                   class_mode = 'categorical',
                                                   shuffle = False,
                                                   batch_size = 32)

In [ ]:
#Menghitung jumlah seluruh dataset
total = train_dataset.samples + val_dataset.samples
total

In [ ]:
#Melihat label apa saja yang digunakan
train_dataset.class_indices

In [ ]:
#Plot gambar dari folder train secara acak
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=2, ncols=4, figsize=(10, 5))#Mendefinisikan ukuran gambar

for i in range(2):
    for j in range(4):
        rand1 = np.random.randint(len(train_dataset))
        rand2 = np.random.randint(10)
        ax[i, j].imshow(train_dataset[rand1][0][rand2])
        ax[i, j].axis('off')
        label = train_dataset[rand1][1][rand2]
        class_label = np.argmax(label)  # Mendapatkan nilai maximum pada label
        if class_label == 0:
            ax[i, j].set_title('Bacterial Spot')
        elif class_label == 1:
            ax[i, j].set_title('Leaf Mold')
        elif class_label == 2:
            ax[i, j].set_title('Tomato Mosaic Virus')
        else:
            ax[i, j].set_title('Healthy')

plt.tight_layout()
plt.show()

In [ ]:
#Mengambil model pretrained MobileNetV2 dari tensorflow keras
from tensorflow.keras.applications import MobileNetV2


pretrained_model = tf.keras.applications.MobileNetV2(input_shape=[224, 224, 3],
                                                     include_top=False,
                                                     weights="imagenet")

pretrained_model.trainable = False

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Dropout, Activation, InputLayer
from tensorflow.keras.models import Model


x = pretrained_model.output

#Menambahkan layer di atas base model
x = Flatten()(x)
x = BatchNormalization()(x)
x = Dense(16, kernel_initializer='he_uniform')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.2)(x)
output_tensor = Dense(10, activation='softmax')(x)

#Membuat model fungsional
model = Model(inputs=pretrained_model.input, outputs=output_tensor)

model.summary()


In [ ]:
#Mendefenisikan fungsi untuk optimiser yaitu dengan metode adam, optimizer berguna untuk mengatur tingkat kecepatan pelatihan
OPT = tf.keras.optimizers.Adam()

model.compile(loss='categorical_crossentropy',#Mendefenisikan fungsi loss
              metrics=['accuracy'],#Mendefenisikan fungsi metrics
              optimizer=OPT)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler
from google.colab import drive

#Hubungkan ke Google Drive
drive.mount('/content/drive')

#Tentukan path untuk menyimpan model terbaik
filepath = '/content/drive/MyDrive/daun_tomat.keras'

#Membuat fungsi checkpoint
checkpoint    = ModelCheckpoint(filepath,
                                monitor = 'val_accuracy',#Berfungsi untuk memonitoring akurasi data validation
                                mode='max',#Mengambil nilai maximum dari hasil yang diperoleh
                                save_best_only=True,#Hanya akan menyimpan model yang terbaik
                                verbose = 1)
#Membuat fungsi earlystopping
earlystopping = EarlyStopping(monitor = 'val_accuracy',
                              mode = 'max' ,
                              patience = 5, #Berfungsi untuk membuat model berhenti apabila tidak ada improvement selama 5 epoch
                              verbose = 1)

#Membuat fungsi learning rate, learning rate akan membesar per sepuluh epoch
def lr_schedule(epoch):
    lr = 1e-6
    if epoch > 10:
        lr = 1e-5
    if epoch > 20:
        lr = 1e-4
    return lr

#Memanggil fungsi learning rate scheduler
lr_scheduler = LearningRateScheduler(lr_schedule)

#Menggabungkan fungsi checkpoint dan learning rate agar dapat digunakan pada pelatihan
callback_list = [checkpoint, lr_scheduler, earlystopping]


In [ ]:
#Melakukan pelatihan model
model_history10=model.fit(train_dataset,
                        validation_data=val_dataset,
                        epochs = 50,
                        callbacks=callback_list,
                        verbose = 1)

In [ ]:
#Plot model loss
plt.plot(model_history10.history['loss'])
plt.plot(model_history10.history['val_loss'])
plt.title('Model Loss in 50 epochs')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left', bbox_to_anchor=(1,1))
plt.show()

In [ ]:
#Plot model akurasi
plt.plot(model_history10.history['accuracy'])
plt.plot(model_history10.history['val_accuracy'])
plt.title('Model Accuracy in 10 epochs')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left', bbox_to_anchor=(1,1))
plt.show()

In [ ]:
#Melihat akurasi,presisi, dan recall model menggunakan data validasi
from sklearn.metrics import classification_report

predictions = model.predict(val_dataset)
true_labels = val_dataset.classes
class_indices = val_dataset.class_indices
class_labels = {v: k for k, v in class_indices.items()}

predicted_labels = np.argmax(predictions, axis=1)
report = classification_report(true_labels, predicted_labels, target_names=list(class_labels.values()))
print("Classification Report:\n", report)

In [ ]:
#Accuracy dan Error Rate
loss, accuracy = model.evaluate(val_dataset)
error_rate = 1 - accuracy

print(f'Accuracy: {accuracy}')
print(f'Error Rate: {error_rate}')

In [ ]:
#Matriks konfusi model dari data validasi
from sklearn.metrics import confusion_matrix
import seaborn as sns
# Membuat matriks konfusi
cm = confusion_matrix(true_labels, predicted_labels)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_labels.values(), yticklabels=class_labels.values())
plt.title('Confusion Matrix')
plt.xlabel('Prediksi')
plt.ylabel('Aktual')
plt.show()

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Import necessary libraries
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import io
import ipywidgets as widgets

# Memuat model yang telah disimpan pada proses training
#model = load_model('/content/drive/MyDrive/daun_tomat.keras')

# Fungsi untuk melakukan prediksi gambar
def predict_image(model, img, class_labels):
    img = img.resize((224, 224))  # Ubah ukuran sesuai model
    img = img_to_array(img)
    img = img / 255.0  # Normalisasi
    img = np.expand_dims(img, axis=0)  # Tambahkan dimensi batch

    # Melakukan prediksi
    predictions = model.predict(img)
    predicted_class_index = np.argmax(predictions)
    predicted_class = class_labels[predicted_class_index]

    return predicted_class

# Daftar nama kelas
class_labels = ['Bacterial Spot', 'Leaf Mold', 'Tomato Mosaic Virus', 'Healthy']

# Membuat widget unggahan file
upload = widgets.FileUpload(accept='image/*', multiple=False)

# Fungsi untuk menangani unggahan file dan prediksi
def on_upload_change(change):
    for name, file_info in upload.value.items():
        image = Image.open(io.BytesIO(file_info['content']))
        plt.imshow(image)
        plt.axis('off')
        plt.show()

        predicted_class = predict_image(model, image, class_labels)
        print("Gambar ini diprediksi sebagai:", predicted_class)

# Memicu fungsi saat ada perubahan pada unggahan
upload.observe(on_upload_change, names='value')

# Menampilkan widget unggahan
display(upload)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileUpload(value={}, accept='image/*', description='Upload')